In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [2]:
def smoothed_mean(X, y, kf, alpha, global_mean=None, columns=None):

    if global_mean is None:
        global_mean = np.mean(y)
        
    if columns is None:
        columns = X.columns
    
    unic_values_X = {}
    for column in columns:
        X[column + '_agr'] = np.NaN
        unic_values_X[column] = set(X[column])
            
    
    for train_indexes, fill_indexes in kf.split(X, y):
        for column in columns:
            for value in unic_values_X[column]:
                slice_train = X.ix[train_indexes][column] == value
                n_rows = slice_train.sum()
                mean_y = 0
                if n_rows > 0:
                    mean_y = y[slice_train[slice_train].index].mean()
                smoothed_mean = (mean_y * n_rows + global_mean * alpha) / (n_rows + alpha)
                
                slice_fill = X.ix[fill_indexes][column] == value
                slice_fill_indexes = slice_fill[slice_fill].index
                X.loc[slice_fill_indexes, column + '_agr'] = smoothed_mean
    return X

In [3]:
number_rows = 10
train_X = pd.DataFrame(np.random.randint(low=10, size=(number_rows,5)), columns=['a', 'b', 'c', 'd', 'e'])
train_y = np.random.binomial(1, 0.7, number_rows)

test_X = pd.DataFrame(np.random.randint(low=10, size=(number_rows,5)), columns=['a', 'b', 'c', 'd', 'e'])
test_y = np.random.binomial(1, 0.7, number_rows)

In [4]:
alpha = 10
global_mean = train_y.mean()
columns = ['c', 'a']

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf_fill = KFold(n_splits=3, shuffle=True, random_state=42)
for train_index, test_index in kf.split(train_X, train_y):
    
    X_train_new = smoothed_mean(X=train_X.ix[train_index], 
                                y=train_y[train_index], 
                                kf=kf_fill,
                                alpha=alpha, 
                                global_mean=global_mean)
    break
    
    """
    print('train_X \n', train_X.ix[train_index])
    print('------')
    print('train_y \n', train_y[train_index])
    print('------')
    print('test_X \n', train_X.ix[test_index])
    print('------')
    print('test_y \n', train_y[test_index])
    print('---------------------------------------------------')
    """
    

In [6]:
X_train_new

,a,b,c,d,e,a_agr,b_agr,c_agr,d_agr,e_agr
0,6,3,7,4,6,0.700000,0.7,0.75,0.700000,0.727273
2,3,7,7,2,5,0.700000,0.7,0.75,0.727273,0.700000
3,4,1,7,5,1,0.666667,0.7,0.75,0.727273,0.700000
4,4,0,9,5,8,0.727273,0.7,0.70,0.727273,0.700000
5,0,9,2,6,3,0.700000,0.7,0.70,0.700000,0.636364
6,8,2,4,2,6,0.700000,0.7,0.70,0.727273,0.727273
7,4,8,6,1,3,0.727273,0.7,0.70,0.700000,0.727273
9,4,1,3,6,7,NaN,NaN,NaN,NaN,NaN
